In [17]:
import os
import sys
import json
import copy
from rv_zep import parse_file
import tabulate

OPCODES = {
    'fld': 0,
    'fsd': 1,
    'fadd': 2,
    'fsub': 3,
    'fmul': 4,
    'fdiv': 5
}

# Define register prefix constants
REG_PREFIXES = {
    'x': 'int',
    'f': 'float'
}


In [8]:
config_file = open("./unit_config.json")
units_configuration = json.load(config_file)

In [6]:
units_configuration['fint']

{'number': 1, 'ex_cycles': 1}

In [14]:
for unit in units:
    print(unit)

fint
fmult
fadd
fdiv


In [35]:
class ScoreBoard:
    def __init__(self, instructions, configuration):

        self.units_configuration = configuration
        self.instruction_queue = instructions
        self.board = []
        self.ops = {}
        self.clock_counter = 0
        self.solution = self.build_solution(self.instruction_queue)
    
    def tick(self):

        self.issue_instruction()
        self.update_board()
        self.clock_counter += 1

        if self.instruction_queue == [] and not self.board_busy():
            return True
        else:
            return False

    def build_board(self):
        #[name][busy][op][fi][fj][fk][qj][qk][rj][rk][instruction_line][state][state_cycles]
        board = []
        base_unit = {'type':None, 'busy':None, 'op':None, 'fi':None, 'fk':None, 'qj':None, 'qk':None, 'rj':None, 'instruction_line':None,'state':None,'ex_cycles':None}

        for unit in self.units_configuration:
            unit_number = self.units_configuration[unit]['number']
            for count in range(unit_number):
                functional_unit = copy.deepcopy(base_unit)
                functional_unit['type'] = unit
                functional_unit['busy'] = False
                functional_unit['ex_cycles'] = unit['ex_cycles']
                board.append(functional_unit)

    def build_solution(self, instruction_list):
        instruction = 0
        issue = 1
        read = 2
        ex = 3
        write = 4

        base_solution = ['Name', None, None, None, None]
        header = copy.deepcopy(base_solution)
        header[issue] = 'Issue'
        header[read] = 'Read'
        header[ex] = 'Ex'
        header[write] = 'Write'
        solution = []
        solution.append(header)

        for current_instruction in instruction_list:
            instruction_solution = copy.deepcopy(base_solution)
            instruction_solution[instruction] = current_instruction['opcode']
            solution.append(instruction_solution)
        return solution

    
    def update_board(self):
        #[name][busy][op][fi][fj][fk][qj][qk][rj][rk][instruction_line][state][state_cycles]

        """ for unit in units_in_board:
            
            can_change_state = check dependencies and ex times
            if can_change_state:
                change_state(self.unit_configuration[unit])
                write_to_solution(self.clock_counter)
            if finished_writeback:
                clean_unit() """
        ...
    
    def issue_instruction(self):

        instruction = self.instruction_queue[0]
        """#check if there are free FUs available
        for unit in self.board.type:
            if available:
                set instruction
                update solution
                break 

        #check if no other issues write to the same destination """
        ...

    def board_busy(self):
        busy = False
        for unit in self.board:
            busy = unit['busy'] or busy
        return busy

    def print_board(self):
        print(tabulate.tabulate(self.board))

    def print_solution(self):
        print(tabulate.tabulate(self.solution))
        ...

    def run(self):
        
        self.done = False
        while not self.done:
            self.tick()
        self.print_solution()
        

In [36]:
config_file = open("./unit_config.json")
configuration = json.load(config_file)
instructions = parse_file('example.s')

board = ScoreBoard(instructions, configuration)
board.print_board()

----  -----  ----  --  -----
Name  Issue  Read  Ex  Write
0
0
5
4
2
1
----  -----  ----  --  -----


In [26]:
configuration

{'fint': {'number': 1, 'ex_cycles': 1},
 'fmult': {'number': 2, 'ex_cycles': 4},
 'fadd': {'number': 1, 'ex_cycles': 2},
 'fdiv': {'number': 1, 'ex_cycles': 10}}

In [29]:
instructions

[{'opcode': 0,
  'rs1': 1,
  'rs1_type': 'int',
  'rs2': 0,
  'rs2_type': None,
  'rd': 1,
  'rd_type': 'float',
  'imm': 0},
 {'opcode': 0,
  'rs1': 1,
  'rs1_type': 'int',
  'rs2': 0,
  'rs2_type': None,
  'rd': 5,
  'rd_type': 'float',
  'imm': 0},
 {'opcode': 5,
  'rs1': 4,
  'rs1_type': 'float',
  'rs2': 5,
  'rs2_type': 'float',
  'rd': 2,
  'rd_type': 'float',
  'imm': None},
 {'opcode': 4,
  'rs1': 8,
  'rs1_type': 'float',
  'rs2': 9,
  'rs2_type': 'float',
  'rd': 4,
  'rd_type': 'float',
  'imm': None},
 {'opcode': 2,
  'rs1': 2,
  'rs1_type': 'float',
  'rs2': 3,
  'rs2_type': 'float',
  'rd': 1,
  'rd_type': 'float',
  'imm': None},
 {'opcode': 1,
  'rs1': 2,
  'rs1_type': 'int',
  'rs2': 4,
  'rs2_type': 'float',
  'rd': 0,
  'rd_type': None,
  'imm': 0}]

In [14]:
a = {'banana':1,'orange':5}

In [15]:
a.get(5)